In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

import ROOT
import pdf_definitions as pdfs

In [ ]:
h = ROOT.TH1F()

In [ ]:

c1 = ROOT.TCanvas()
c1.cd()
h.Draw()

c1.Update()

#c1.Show()

# rf202

In [ ]:
# Declare observable x
x = ROOT.RooRealVar("x", "x", 0, 10)

# Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and
# their parameters
mean = ROOT.RooRealVar("mean", "mean of gaussians", 5)
sigma1 = ROOT.RooRealVar("sigma1", "width of gaussians", 0.5)
sigma2 = ROOT.RooRealVar("sigma2", "width of gaussians", 1)

sig1 = ROOT.RooGaussian("sig1", "Signal component 1", x, mean, sigma1)
sig2 = ROOT.RooGaussian("sig2", "Signal component 2", x, mean, sigma2)

# Build Chebychev polynomial p.d.f.
a0 = ROOT.RooRealVar("a0", "a0", 0.5, 0., 1.)
a1 = ROOT.RooRealVar("a1", "a1", -0.2, 0., 1.)
bkg = ROOT.RooChebychev("bkg", "Background", x, ROOT.RooArgList(a0, a1))

# Sum the signal components into a composite signal p.d.f.
sig1frac = ROOT.RooRealVar(
    "sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.)
sig = ROOT.RooAddPdf(
    "sig", "Signal", ROOT.RooArgList(sig1, sig2), ROOT.RooArgList(sig1frac))



In [ ]:
# Method 1 - Construct extended composite model
# -------------------------------------------------------------------

# Sum the composite signal and background into an extended pdf
# nsig*sig+nbkg*bkg
nsig = ROOT.RooRealVar("nsig", "number of signal events", 500, 0., 10000)
nbkg = ROOT.RooRealVar(
    "nbkg", "number of background events", 500, 0, 10000)
model = ROOT.RooAddPdf(
    "model",
    "(g1+g2)+a",
    ROOT.RooArgList(
        bkg,
        sig),
    ROOT.RooArgList(
        nbkg,
        nsig))

# Sample, fit and plot extended model
# ---------------------------------------------------------------------

# Generate a data sample of expected number events in x from model
# = model.expectedEvents() = nsig+nbkg
data = model.generate(ROOT.RooArgSet(x))

# Fit model to data, ML term automatically included
model.fitTo(data)

# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe = x.frame(ROOT.RooFit.Title("extended ML fit example"))
data.plotOn(xframe)
model.plotOn(xframe, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(bkg)
model.plotOn(
    xframe, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background+sig2 components of model with a dotted line
ras_bkg_sig2 = ROOT.RooArgSet(bkg, sig2)
model.plotOn(
    xframe, ROOT.RooFit.Components(ras_bkg_sig2), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))


In [ ]:
# Print structure of composite p.d.f.
model.Print("t")


# Method 2 - Construct extended components first
# ---------------------------------------------------------------------

# Associated nsig/nbkg as expected number of events with sig/bkg
esig = ROOT.RooExtendPdf("esig", "extended signal p.d.f", sig, nsig)
ebkg = ROOT.RooExtendPdf("ebkg", "extended background p.d.f", bkg, nbkg)

# Sum extended components without coefs
# -------------------------------------------------------------------------

# Construct sum of two extended p.d.f. (no coefficients required)
model2 = ROOT.RooAddPdf("model2", "(g1+g2)+a", ROOT.RooArgList(ebkg, esig))

# Draw the frame on the canvas
c = ROOT.TCanvas("rf202_extendedmlfit", "rf202_extendedmlfit", 600, 600)
ROOT.gPad.SetLeftMargin(0.15)
xframe.GetYaxis().SetTitleOffset(1.4)
xframe.Draw()

c.Update()

c.SaveAs("rf202_extendedmlfit.png")


# From ChatGPT

In [ ]:
c = ROOT.TCanvas("testingfits", "testingfits")#"fit", "fit", 600, 600)

In [ ]:
#df = pd.read_parquet('mes_deltae_background_AFTER_FINAL_CUTS.parquet')
#df

In [ ]:
mes_lo, mes_hi = 5.2,5.3
deltae_lo, deltae_hi = -0.2,0.2

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])


x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
y.setRange("SB3", -0.075, 0.075)



In [ ]:
#dim = 2

#dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x,y, data_z,
#:dim, use_double_cb, workspace)

pars_bkg, bkg_pdf = pdfs.argus_in_x(x)
pars_sig, sig_pdf = pdfs.crystal_barrel_x(x)

conv_factor_fit  = ROOT.RooRealVar("conv_factor_fit", "Conversion factor (fit)",10) # Conversion factor, fit
branching_fraction = ROOT.RooRealVar("branching_fraction","Branching fraction",1)


nbkg = ROOT.RooRealVar("nbkg","# bkg events,",10)
nsig = ROOT.RooFormulaVar("nsig","conv_factor_fit*branching_fraction",ROOT.RooArgList(convmes_lo, mes_hi = 5.2,5.3))
deltae_lo, deltae_hi = -0.2,0.2

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])


x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
y.setRange("SB3", -0.075, 0.075)
_factor_fit,branching_fraction))


total = ROOT.RooAddPdf("total","sig_pdf + bkg_pdf", ROOT.RooArgList(sig_pdf, bkg_pdf), ROOT.RooArgList(nsig, nbkg))

pars = [nbkg, nsig]
pars += pars_sig
pars += pars_bkg


In [ ]:
conv_factor_fit.setVal(4)
nbkg.setVal(9)

data = total.generate(ROOT.RooArgSet(x))
conv_factor_fit.setConstant(False)

nbkg.setConstant(False)

# Fit model to data, ML term automatically included
'''
result = total.fitTo(data, ROOT.RooFit.Extended(ROOT.kTRUE),
                                         ROOT.RooFit.Save(ROOT.kTRUE),
                                         ROOT.RooFit.Strategy(2),
                                         ROOT.RooFit.PrintLevel(-1))
'''
nll = ROOT.RooNLLVar("nll","nll",total,data,ROOT.RooFit.Extended(ROOT.kTRUE))
#fit_func = RooFormulaVar("fit_func","nll + log_gc",RooArgList(nll,pars_d["log_gc"]))
fit_func = ROOT.RooFormulaVar("fit_func","nll",ROOT.RooArgList(nll))

m = ROOT.RooMinimizer(fit_func)
m.setVerbose(ROOT.kFALSE)
m.migrad()
m.hesse()
result = m.save()



result.Print("v")



In [ ]:
result.Print('v')

In [ ]:
del c
c = ROOT.TCanvas("testingfits", "testingfits", 600, 300)#"fit", "fit", 600, 600)
# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(25))
data.plotOn(xframe)
total.plotOn(xframe, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(bkg_pdf)
total.plotOn(
    xframe, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_sig = ROOT.RooArgSet(sig_pdf)
total.plotOn(
    xframe, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Draw the frame on the canvas

ROOT.gPad.SetLeftMargin(0.15)
xframe.GetYaxis().SetTitleOffset(1.4)
xframe.Draw()



#c.Update()
#ROOT.gPad
c.Draw()
#ROOT.gPad.Draw()
#ROOT.gPad.Update()
#c.Draw()

#c.SaveAs("testfit.png")


In [ ]:
def read_file_return_dataset(df, x, y, region_definitions=None, dim=2, max_events=1e9, index=0):
    
    nevents=0
    name = f"dataset_{index}"
    ds = ROOT.RooDataSet(name, name, ROOT.RooArgSet(x,y) )

    # mES ranges
    x_lo = region_definitions['fitting MES'][0]
    x_hi = region_definitions['fitting MES'][1]
    # DeltaE ranges
    y_lo = region_definitions['fitting DeltaE'][0]
    y_hi = region_definitions['fitting DeltaE'][1]

    print("Data ranges...")
    print(x_lo, x_hi)
    print(y_lo, y_hi)
    
    mes, deltae = df['BpostFitMes'].values, df['BpostFitDeltaE'].values

    for m,de in zip(mes, deltae):
        x.setVal(m)
        y.setVal(de)

        if nevents<max_events:
            #print "%3.3f %3.3f %3.3f" % (x.getVal(), y.getVal(), z.getVal())
            #print "\t%3.3f %3.3f" % (x_lo, x_hi)
            #print "\t%3.3f %3.3f" % (y_lo, y_hi)
            #print "\t%3.3f %3.3f" % (z_lo, z_hi)
            #print "here"

            if x.getVal()>x_lo and x.getVal()<x_hi and \
               y.getVal()>y_lo and y.getVal()<y_hi:
                #if y.getVal()>y_lo and y.getVal()<y_hi:
                # Run this check otherwise the fit won't converge.
                #print "Adding...."
                ds.add(ROOT.RooArgSet(x,y))
                nevents += 1
                #print "Added event --------------"
            '''
            else:
                print "NOT ADDING EVENT ========+++++++++===========++++++++++++"
            '''
        else:
            #print "hi"
            break

    #print "Here is hte nevents: %d %f" % (nevents, max_events)
    return ds


In [ ]:
#df_sp = pd.read_parquet("DATAFRAME_SP.parquet")
#df_col = pd.read_parquet("DATAFRAME_COL.parquet")

df_sp = pd.read_parquet("DATAFRAME_SP_BNC_ALL.parquet")
df_col = pd.read_parquet("DATAFRAME_COL_BNC_ALL.parquet")

In [ ]:
#mask = (df_sp['spmode']=='998')
mask = (df_sp['used_in_bkg_train'] == False)
mask &= mask & (df_sp['used_in_sig_train'] == False)
mask &= mask & (df_sp['cut_2']==True) & (df_sp['cut_3']==True)  & (df_sp['cut_4']==True)
mask = mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
mask &= mask & (df_sp['proba']>0.97)

mask_sp = (df_sp['spmode']=='998')
df_tmp1 = df_sp[mask & mask_sp]

mask_sp = (df_sp['spmode']=='-999')
df_tmp2 = df_sp[mask & mask_sp].sample(n=4)

df_tmp = pd.concat([df_tmp1, df_tmp2])

#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

In [ ]:
ds.Print("v")

In [ ]:
ds_x = ds.reduce(x)

ds_x.Print("v")

In [ ]:
ds_x.sumEntries()

In [ ]:
#dim = 2

#dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x,y, data_z,
#:dim, use_double_cb, workspace)

pars_bkg, bkg_pdf = pdfs.argus_in_x(x)
pars_sig, sig_pdf = pdfs.crystal_barrel_x(x)

conv_factor_fit  = ROOT.RooRealVar("conv_factor_fit", "Conversion factor (fit)",10) # Conversion factor, fit
branching_fraction = ROOT.RooRealVar("branching_fraction","Branching fraction",1)

nbkg = ROOT.RooRealVar("nbkg","# bkg events,",10)
nsig = ROOT.RooFormulaVar("nsig","conv_factor_fit*branching_fraction",ROOT.RooArgList(conv_factor_fit,branching_fraction))

total = ROOT.RooAddPdf("total","sig_pdf + bkg_pdf", ROOT.RooArgList(sig_pdf, bkg_pdf), ROOT.RooArgList(nsig, nbkg))

pars = [nbkg, nsig]
pars += pars_sig
pars += pars_bkg

conv_factor_fit.setVal(0.1)
nbkg.setVal(ds_x.sumEntries())
conv_factor_fit.setConstant(False)
nbkg.setConstant(False)

nll = ROOT.RooNLLVar("nll","nll",total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))
#fit_func = RooFormulaVar("fit_func","nll + log_gc",RooArgList(nll,pars_d["log_gc"]))
fit_func = ROOT.RooFormulaVar("fit_func","nll",ROOT.RooArgList(nll))

m = ROOT.RooMinimizer(fit_func)
m.setVerbose(ROOT.kFALSE)
m.migrad()
m.hesse()
result = m.save()

result.Print("v")


xframe2 = x.frame(Title="Gaussian pdf with data")  # RooPlot

#del c2
c2 = ROOT.TCanvas("testingfits", "testingfits", 600, 300)#"fit", "fit", 600, 600)

ds.plotOn(xframe2)


total.plotOn(xframe2, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(bkg_pdf)
total.plotOn(
    xframe2, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_sig = ROOT.RooArgSet(sig_pdf)
total.plotOn(
    xframe2, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))




xframe2.Draw()
#c.Update()
#ROOT.gPad
c2.Draw()

# Full fit maybe?


## Read in the data files

In [ ]:
#BNC_tag = ""
#BNC_bool = False
#ntrain_tag = 'nsig_20000_nbkg_20000'

BNC_tag = "_BNC"
BNC_bool = True
#ntrain_tag = 'nsig_30000_nbkg_30000'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial0'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial1'
#ntrain_tag = 'features_2_nsig_40000_nbkg_40000_trial0'
ntrain_tag = 'features_4_nsig_60000_nbkg_60000_trial1'


# Read in the dfs
infilename_sp = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
#df_sp = pd.read_parquet(infilename)

infilename_col = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
#df_col = pd.read_parquet(infilename)



In [ ]:
df_sp = pd.read_parquet(infilename_sp)
df_col = pd.read_parquet(infilename_col)


#df_sp = pd.read_parquet("DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000_BNC.pkl")
#df_col = pd.read_parquet("DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000_BNC.pkl")

#df_sp = pd.read_parquet("DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")
#df_col = pd.read_parquet("DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")

In [ ]:

mes_lo, mes_hi = 5.2,5.3
deltae_lo, deltae_hi = -0.2,0.2

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])


x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
y.setRange("SB3", -0.075, 0.075)



In [ ]:
ds_x = ds.reduce(x)

In [ ]:
mypars = []

dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x, ds_x)#, use_double_cb, workspace)

mypars += dum_pars


total.Print("v")

In [ ]:
# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f


In [ ]:
for p in pars_d.keys():
    print(f"{p} ---------------------")
    #pars_d[p].Print("v")

In [ ]:
df_sp = pd.read_parquet("DATAFRAME_SP_BNC_ALL.parquet")
df_col = pd.read_parquet("DATAFRAME_COL_BNC_ALL.parquet")

In [ ]:
#mask = (df_sp['spmode']=='998')
mask = (df_sp['used_in_bkg_train'] == False)
mask &= mask & (df_sp['used_in_sig_train'] == False)
mask &= mask & (df_sp['cut_2']==True) & (df_sp['cut_3']==True) & (df_sp['cut_4']==True)
mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
mask &= mask & (df_sp['proba']>0.90)

mask_sp = (df_sp['spmode']=='998')
df_tmp1 = df_sp[mask & mask_sp]

mask_sp = (df_sp['spmode']=='-999')
df_tmp2 = df_sp[mask & mask_sp].sample(n=5)

df_tmp = pd.concat([df_tmp1, df_tmp2])
#df_tmp = df_tmp1

#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")

In [ ]:
mypars = []
dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x, ds_x)#, use_double_cb, workspace)
mypars += dum_pars
total.Print("v")

# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f



# Background

pars_d["nbkg"].setVal(ds_x.sumEntries())
pars_d["nbkg"].setConstant(False)

pars_d["argpar"].setVal(-23.5)
pars_d["argpar"].setConstant(True)

pars_d["cutoff"].setVal(5.29)
pars_d["cutoff"].setConstant(True)

## Signal

pars_d["nCB"].setVal(1.0)
pars_d["nCB"].setConstant(True)

#pars_d["conv_factor_calc"].setVal(43.)
pars_d["conv_factor_calc"].setVal(13.)
pars_d["conv_factor_calc"].setConstant(True)

#pars_d["conv_factor_fit"].setVal(43.)
pars_d["conv_factor_fit"].setVal(13.)
pars_d["conv_factor_fit"].setConstant(False)

pars_d["conv_factor_err"].setVal(500)
pars_d["conv_factor_err"].setConstant(True)

#pars_d["branching_fraction"].setVal(0.58)
pars_d["branching_fraction"].setVal(12.2)
pars_d["branching_fraction"].setConstant(True)

pars_d["meanCB"].setVal(5.279)
pars_d["meanCB"].setConstant(True)

pars_d["sigmaCB"].setVal(0.0028)
pars_d["sigmaCB"].setConstant(True)

pars_d["alphaCB"].setVal(2.000)
pars_d["alphaCB"].setConstant(True)

# Bkg only

total.Print("v")

# Create the NLL for the fit
nll = ROOT.RooNLLVar("nll","nll", total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))
fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc",ROOT.RooArgList(nll,pars_d["log_gc"]))

m = ROOT.RooMinimizer(fit_func)

m.setVerbose(ROOT.kFALSE)

m.migrad()
m.hesse()
fit_results = m.save()


fit_results.Print("v")
mynsig = pars_d["conv_factor_fit"].getVal() * pars_d["branching_fraction"].getVal()
print(f"mynsig: {mynsig}    nsig: {pars_d['nsig'].getVal()}")



In [ ]:
fit_results.Print("v")
print()
mynsig = pars_d["conv_factor_fit"].getVal() * pars_d["branching_fraction"].getVal()
print(f"mynsig: {mynsig}    nsig: {pars_d['nsig'].getVal()}")


In [ ]:
#del c
c4 = ROOT.TCanvas("testingfits4", "testingfits4", 600, 300)#"fit", "fit", 600, 600)
# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe4 = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(25))
ds_x.plotOn(xframe4)
total.plotOn(xframe4, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(bkg_pdf)
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_sig = ROOT.RooArgSet(sig_pdf)
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Draw the frame on the canvas

ROOT.gPad.SetLeftMargin(0.15)
xframe4.GetYaxis().SetTitleOffset(1.4)
xframe4.Draw()



#c.Update()
#ROOT.gPad
c4.Draw()
#ROOT.gPad.Draw()
#ROOT.gPad.Update()
#c.Draw()

#c.SaveAs("testfit.png")


# Start here

## Background

In [ ]:
#BNC_tag = ""
#BNC_bool = False
#ntrain_tag = 'nsig_20000_nbkg_20000'

BNC_tag = "_BNC"
BNC_bool = True
#ntrain_tag = 'nsig_30000_nbkg_30000'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial0'
#ntrain_tag = 'nsig_40000_nbkg_40000_trial1'
#ntrain_tag = 'features_2_nsig_40000_nbkg_40000_trial0'
ntrain_tag = 'features_4_nsig_60000_nbkg_60000_trial3'


# Read in the dfs
infilename_sp = f"DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
#df_sp = pd.read_parquet(infilename)

infilename_col = f"DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_{ntrain_tag}{BNC_tag}.pkl"
#df_col = pd.read_parquet(infilename)



In [ ]:
df_sp = pd.read_parquet(infilename_sp)
df_col = pd.read_parquet(infilename_col)

#df_sp = pd.read_parquet("DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")
#df_col = pd.read_parquet("DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")


proba_cut = 0.82

In [ ]:
# BNC
mes_lo, mes_hi = 5.2,5.3

# Trying this for BNV
#mes_lo, mes_hi = 5.21,5.3

#deltae_lo, deltae_hi = -0.2,0.2
deltae_lo, deltae_hi = -0.07,0.07

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])


x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
#y.setRange("SB3", -0.075, 0.075)
y.setRange("SB3", -0.05, 0.05)



In [ ]:
# BNC
mask = (df_sp['used_in_bkg_train'] == False)
mask &= mask & (df_sp['used_in_sig_train'] == False)
mask &= mask & (df_sp['cut_2']==True) & (df_sp['cut_3']==True) & (df_sp['cut_4']==True)
mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
mask &= mask & (df_sp['proba']>proba_cut)

mask_sp = (df_sp['spmode']!='-999')
df_tmp1 = df_sp[mask & mask_sp]


# BNV
'''
#mask = (df_sp['spmode']=='998')
mask = (df_sp['used_in_bkg_train'] == False)
mask &= mask & (df_sp['cut_-1']==True)
mask &= mask & (df_sp['BpostFitDeltaE']<0.07) & (df_sp['BpostFitDeltaE']>-0.07)
mask &= mask & (df_sp['proba']>0.84)
'''

mask_sp = (df_sp['spmode']=='998')
df_tmp1 = df_sp[mask & mask_sp]



#mask_sp = (df_sp['spmode']=='-999')
#df_tmp2 = df_sp[mask & mask_sp].sample(n=4)

#df_tmp = pd.concat([df_tmp1, df_tmp2])
df_tmp = df_tmp1

#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")

In [ ]:
#df_tmp

mypars = []
dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x, ds_x)#, use_double_cb, workspace)
mypars += dum_pars
total.Print("v")

# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f


In [ ]:
for f in sub_funcs.keys():
    sub_funcs[f].Print()

In [ ]:
pars_d["nbkg"].setVal(ds_x.sumEntries())
pars_d["nbkg"].setConstant(False)

pars_d["argpar"].setVal(-23.5)
pars_d["argpar"].setConstant(False)

pars_d["cutoff"].setVal(5.29)
pars_d["cutoff"].setConstant(False)



# Bkg only
total = ROOT.RooExtendPdf("fit_func", "Extended function for background", sub_funcs["bkg_pdf"], pars_d["nbkg"])

total.Print("v")

# Create the NLL for the fit
nll = ROOT.RooNLLVar("nll","nll",total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))
fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc", ROOT.RooArgList(nll,pars_d["log_gc"]))

m = ROOT.RooMinimizer(fit_func)

m.setVerbose(ROOT.kFALSE)


m.migrad()
m.hesse()
fit_results = m.save()


fit_results.Print("v")


In [ ]:
xframe2 = x.frame(ROOT.RooFit.Title("Gaussian pdf with data"),   ROOT.RooFit.Bins(50))  # RooPlot
#xframe2 = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(25))

#del c2
c2 = ROOT.TCanvas("testingfits", "testingfits", 600, 300)#"fit", "fit", 600, 600)

ds_x.plotOn(xframe2)

total.plotOn(xframe2, ROOT.RooFit.Normalization(1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
total.plotOn(
    xframe2, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

## Overlay the background component of model with a dashed line
#ras_sig = ROOT.RooArgSet(sig_pdf)
#total.plotOn(
#    xframe2, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
#        ROOT.kDotted), ROOT.RooFit.Normalization(
#            1.0, ROOT.RooAbsReal.RelativeExpected))

xframe2.Draw()
c2.Draw()

# Signal

In [ ]:
# Bkg only
total = ROOT.RooExtendPdf("fit_func", "Extended function for signal", sub_funcs["sig_pdf"], pars_d["nsig"])

total.Print("v")

In [ ]:
# BNC
#'''
mask = (df_sp['used_in_bkg_train'] == False)
mask &= mask & (df_sp['used_in_sig_train'] == False)
mask &= mask & (df_sp['cut_2']==True) & (df_sp['cut_3']==True) & (df_sp['cut_4']==True)
mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
mask &= mask & (df_sp['proba']>proba_cut)

mask_sp = (df_sp['spmode']=='-999')
df_tmp1 = df_sp[mask & mask_sp]
#'''

# BNV
'''
mask = (df_sp['used_in_sig_train'] == False)
mask &= mask & (df_sp['cut_-1']==True)
mask &= mask & (df_sp['BpostFitDeltaE']<0.05) & (df_sp['BpostFitDeltaE']>-0.05)
mask &= mask & (df_sp['proba']>0.10)

mask_sp = (df_sp['spmode']=='-999')
df_tmp1 = df_sp[mask & mask_sp]
'''

#mask_sp = (df_sp['spmode']=='-999')
#df_tmp2 = df_sp[mask & mask_sp].sample(n=4)

#df_tmp = pd.concat([df_tmp1, df_tmp2])
df_tmp = df_tmp1

#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")

In [ ]:
pars_d.keys()

In [ ]:
'''
pars_d["nbkg"].setVal(ds_x.sumEntries())
pars_d["nbkg"].setConstant(False)

pars_d["argpar"].setVal(-23.5)
pars_d["argpar"].setConstant(False)

pars_d["cutoff"].setVal(5.29)
pars_d["cutoff"].setConstant(False)


# Bkg only
total = ROOT.RooExtendPdf("fit_func", "Extended function for background", sub_funcs["bkg_pdf"], pars_d["nbkg"])

total.Print("v")

# Create the NLL for the fit
nll = ROOT.RooNLLVar("nll","nll",total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))
fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc", ROOT.RooArgList(nll,pars_d["log_gc"]))

m = ROOT.RooMinimizer(fit_func)

m.setVerbose(ROOT.kFALSE)


m.migrad()
m.hesse()
fit_results = m.save()


fit_results.Print("v")
'''

In [ ]:
# Create the NLL for the fit

print(f"nevents: {ds_x.sumEntries()}")

pars_d["nCB"].setVal(1.0)
pars_d["nCB"].setConstant(True)

#pars_d["conv_factor_calc"].setVal(43.)
pars_d["conv_factor_calc"].setVal(13.)
pars_d["conv_factor_calc"].setConstant(True)

#pars_d["conv_factor_fit"].setVal(43.)
pars_d["conv_factor_fit"].setVal(13.)
pars_d["conv_factor_fit"].setConstant(False)

pars_d["conv_factor_err"].setVal(500)
pars_d["conv_factor_err"].setConstant(True)

#pars_d["branching_fraction"].setVal(0.58)
pars_d["branching_fraction"].setVal(12.2)
pars_d["branching_fraction"].setConstant(True)

pars_d["meanCB"].setVal(5.279)
pars_d["meanCB"].setConstant(True)

pars_d["sigmaCB"].setVal(0.0028)
pars_d["sigmaCB"].setConstant(False)

pars_d["alphaCB"].setVal(2.000)
pars_d["alphaCB"].setConstant(False)

nll = ROOT.RooNLLVar("nll","nll", total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))

fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc", ROOT.RooArgList(nll,pars_d["log_gc"]))

m = ROOT.RooMinimizer(fit_func)

m.setVerbose(ROOT.kTRUE)
m.setMaxFunctionCalls(1000)
m.setMaxIterations(1000)


m.migrad()

print("After migrad")
fit_results = m.save()
fit_results.Print("v")

m.hesse()
fit_results = m.save()
fit_results.Print("v")

mynsig = pars_d["conv_factor_fit"].getVal() * pars_d["branching_fraction"].getVal()
print(f"mynsig: {mynsig}    nsig: {pars_d['nsig'].getVal()}")


fit_results.Print("v")


In [ ]:
pars_d['nsig'].Print("v")

print(pars_d["nsig"].evaluate())



In [ ]:
xframe3 = x.frame(ROOT.RooFit.Title("Signal"),   ROOT.RooFit.Bins(20))  # RooPlot

#del c2
c3 = ROOT.TCanvas("testingfits", "testingfits", 600, 300)#"fit", "fit", 600, 600)

ds_x.plotOn(xframe3)

total.plotOn(xframe3, ROOT.RooFit.Normalization(1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
#ras_bkg = ROOT.RooArgSet(bkg_pdf)
#total.plotOn(
#    xframe2, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
#        ROOT.kDashed), ROOT.RooFit.Normalization(
#            1.0, ROOT.RooAbsReal.RelativeExpected))

## Overlay the signal component of model with a dashed line
ras_sig = ROOT.RooArgSet(sub_funcs['sig_pdf'])
total.plotOn(
    xframe3, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

xframe3.Draw()
c3.Draw()

# Full fit?

In [ ]:
df_sp = pd.read_parquet(infilename_sp)
df_col = pd.read_parquet(infilename_col)

#df_sp = pd.read_parquet("DATAFRAME_SP_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")
#df_col = pd.read_parquet("DATAFRAME_COL_MODEL_MLPClassifier_CUTS_1_2_3_nsig_20000_nbkg_20000.pkl")

In [ ]:
mes_lo, mes_hi = 5.2,5.3

# Trying this for BNV mode
#mes_lo, mes_hi = 5.21,5.3
#deltae_lo, deltae_hi = -0.07,0.07

# BNC
#deltae_lo, deltae_hi = -0.2,0.2
deltae_lo, deltae_hi = -0.05,0.05

x,y = pdfs.build_xy([[mes_lo, mes_hi], [deltae_lo, deltae_hi]])


x.setRange("FULL",mes_lo,mes_hi)
y.setRange("FULL",deltae_lo, deltae_hi)

x.setRange("SIGNAL",5.25, mes_hi)
y.setRange("SIGNAL",-0.10, 0.1)

# Sideband 1 region
x.setRange("SB1", mes_lo,  mes_hi)
y.setRange("SB1", 0.075, deltae_hi)

# Sideband 2 region
x.setRange("SB2",  mes_lo,  mes_hi)
y.setRange("SB2", deltae_lo, -0.075)

# Sideband 3 region
x.setRange("SB3",  mes_lo,  5.27)
#y.setRange("SB3", -0.075, 0.075)
y.setRange("SB3", -0.05, 0.05)



In [ ]:
ds_x = ds.reduce(x)

In [ ]:
mypars = []

dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x, ds_x)#, use_double_cb, workspace)

mypars += dum_pars


total.Print("v")

In [ ]:
# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f


In [ ]:
for p in pars_d.keys():
    print(f"{p} ---------------------")
    #pars_d[p].Print("v")

In [ ]:
#df_sp = pd.read_parquet("DATAFRAME_SP_BNC_ALL.parquet")
#df_col = pd.read_parquet("DATAFRAME_COL_BNC_ALL.parquet")

In [ ]:
#############################################################
# SP
#############################################################
#'''
# BNC
'''
df_for_fit = df_sp

#mask = (df_sp['spmode']=='998')
mask = (df_for_fit['used_in_bkg_train'] == False)
mask &= mask & (df_for_fit['used_in_sig_train'] == False)
mask &= mask & (df_for_fit['cut_2']==True) & (df_for_fit['cut_3']==True) & (df_for_fit['cut_4']==True)
mask &= mask & (df_for_fit['BpostFitDeltaE']<0.05) & (df_for_fit['BpostFitDeltaE']>-0.05)
mask &= mask & (df_for_fit['proba']>0.97)

mask_sp = (df_for_fit['spmode']=='998')
df_tmp1 = df_for_fit[mask & mask_sp]

mask_sp = (df_for_fit['spmode']=='-999')
df_tmp2 = df_for_fit[mask & mask_sp].sample(n=8)

df_tmp = pd.concat([df_tmp1, df_tmp2])
#df_tmp = df_tmp1
'''
# BNV
'''
df_for_fit = df_sp

#mask = (df_sp['spmode']=='998')
mask = (df_for_fit['used_in_bkg_train'] == False)
mask &= mask & (df_for_fit['used_in_sig_train'] == False)
mask &= mask & (df_for_fit['cut_-1']==True)
mask &= mask & (df_for_fit['BpostFitDeltaE']<0.07) & (df_for_fit['BpostFitDeltaE']>-0.07)
mask &= mask & (df_for_fit['proba']>0.84)

mask_sp = (df_for_fit['spmode']=='998')
df_tmp1 = df_for_fit[mask & mask_sp].sample(6)

mask_sp = (df_for_fit['spmode']=='-999')
df_tmp2 = df_for_fit[mask & mask_sp].sample(n=4)

df_tmp = pd.concat([df_tmp1, df_tmp2])
'''


#'''
#############################################################
# Collision
#############################################################

#'''
df_for_fit = df_col

# BNC
#proba_cut = 0.80
mask = (df_for_fit['cut_2']==True) & (df_for_fit['cut_3']==True) & (df_for_fit['cut_4']==True)
mask &= mask & (df_for_fit['BpostFitDeltaE']<0.07) & (df_for_fit['BpostFitDeltaE']>-0.07)
mask &= mask & (df_for_fit['proba']>proba_cut)

# BNV
#mask = (df_for_fit['cut_-1']==True)
#mask &= mask & (df_for_fit['BpostFitDeltaE']<0.07) & (df_for_fit['BpostFitDeltaE']>-0.07)
#mask &= mask & (df_for_fit['proba']>0.87)

df_tmp = df_for_fit[mask]
#df_tmp = df_tmp1
#'''


#ds = read_file_return_dataset(df_sp[mask], x, y, region_definitions=region_definitions)
ds = read_file_return_dataset(df_tmp, x, y, region_definitions=region_definitions)

ds_x = ds.reduce(x)

ds_x.Print("v")

In [ ]:
#auto mu = (RooRealVar*)params.find("mu");
#mu->setValueDirty();

# Do I need to do this? 
# https://github.com/root-project/root/issues/7659

pars_d['conv_factor_fit'].setValueDirty()

In [ ]:

# Can we reduce the data for just sidebands?
#reduced_data = ds.reduce(ROOT.RooFit.CutRange("SB1"))
#reduced_data.append(ds.reduce(ROOT.RooFit.CutRange("SB2")))
#reduced_data.append(ds.reduce(ROOT.RooFit.CutRange("SB3")))
reduced_data = ds.reduce(ROOT.RooFit.CutRange("SB3"))

reduced_data_x = reduced_data.reduce(x)


# Background
pars_d["nbkg"].setVal(reduced_data_x.sumEntries())
pars_d["argpar"].setVal(-23.5)
pars_d["cutoff"].setVal(5.29)

pars_d["nbkg"].setConstant(False)
pars_d["argpar"].setConstant(False)
pars_d["cutoff"].setConstant(True)

## Signal

# BNC
pars_d["conv_factor_calc"].setVal(2.333e7)
pars_d["conv_factor_fit"].setVal(2.333e7)
pars_d["conv_factor_err"].setVal(7.551e5)
pars_d["branching_fraction"].setVal(0)

pars_d["nCB"].setVal(1.0)
pars_d["meanCB"].setVal(5.279)
pars_d["sigmaCB"].setVal(2.7035e-03)
pars_d["alphaCB"].setVal(16.27)

pars_d["nCB"].setConstant(True)
pars_d["conv_factor_calc"].setConstant(True)
pars_d["conv_factor_fit"].setConstant(True)
pars_d["conv_factor_err"].setConstant(True)
pars_d["branching_fraction"].setConstant(True)
pars_d["meanCB"].setConstant(True)
pars_d["sigmaCB"].setConstant(True)
pars_d["alphaCB"].setConstant(True)



# Create the NLL for the fit
nll = ROOT.RooNLLVar("nll","nll", total, reduced_data_x, ROOT.RooFit.Extended(ROOT.kTRUE))
#fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc",ROOT.RooArgList(nll,pars_d["log_gc"]))

# Should I use RooAddition as mentioned here
# https://root.cern/doc/v612/classRooMinimizer.html
fit_func = ROOT.RooAddition("fit_func","sum of nll and log_gc",ROOT.RooArgList(nll,pars_d["log_gc"]))
# No gives same result with no asymmetric uncertainties

nll.setValueDirty()
fit_func.setValueDirty()

print("Setting up the minimizer..................................")
m = ROOT.RooMinimizer(fit_func)
#m = ROOT.RooMinimizer(nll)

# For NLL?
m.setErrorLevel(0.5)
m.setVerbose(ROOT.kTRUE)
m.setStrategy(2)

print("Helping get the initial step size.........................")
m.hesse()

print("Performing the minimizations..............................")
m.migrad()

print("Calculating the uncertainties..............................")
m.hesse()

fit_results = m.save()


fit_results.Print("v")


In [ ]:
#################################
# Reduced
########################################

#del c
c4 = ROOT.TCanvas("testingfits4", "testingfits4", 600, 300)#"fit", "fit", 600, 600)
# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe4 = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(25))
reduced_data_x.plotOn(xframe4)
total.plotOn(xframe4, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_sig = ROOT.RooArgSet(sub_funcs['sig_pdf'])
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.LineColor("r") , ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Draw the frame on the canvas

ROOT.gPad.SetLeftMargin(0.15)
xframe4.GetYaxis().SetTitleOffset(1.4)
xframe4.Draw()



#c.Update()
#ROOT.gPad
c4.Draw()
#ROOT.gPad.Draw()
#ROOT.gPad.Update()
#c.Draw()

c4.SaveAs("testfit_BNC.png")



In [ ]:
mypars = []
dum_pars, sub_funcs_list, total = pdfs.tot_PDF(x, ds_x)#, use_double_cb, workspace)
mypars += dum_pars
total.Print("v")

# Create a dictionary of the pars
# We'll use this later if we need to print stuff out.
#print mypars
pars_d = {}
for p in mypars:
    pars_d[p.GetName()] = p

# Create a dictionary of extra functions
# We'll use this later if we need to print stuff out.
#print sub_funcs_list
sub_funcs = {}
for f in sub_funcs_list:
    sub_funcs[f.GetName()] = f






# Background
pars_d["nbkg"].setVal(ds_x.sumEntries())
#pars_d["argpar"].setVal(-23.5)
#pars_d["cutoff"].setVal(5.29)

pars_d["nbkg"].setConstant(False)
pars_d["argpar"].setConstant(False)
pars_d["cutoff"].setConstant(True)

## Signal

# BNC
pars_d["conv_factor_calc"].setVal(2.333e7)
pars_d["conv_factor_fit"].setVal(2.333e7)
pars_d["conv_factor_err"].setVal(7.551e5)
pars_d["branching_fraction"].setVal(2.0e-7)

pars_d["nCB"].setVal(1.0)
pars_d["meanCB"].setVal(5.279)
pars_d["sigmaCB"].setVal(2.7035e-03)
pars_d["alphaCB"].setVal(16.27)

pars_d["nCB"].setConstant(True)
pars_d["conv_factor_calc"].setConstant(True)
pars_d["conv_factor_fit"].setConstant(False)
pars_d["conv_factor_err"].setConstant(True)
pars_d["branching_fraction"].setConstant(False)
pars_d["meanCB"].setConstant(True)
pars_d["sigmaCB"].setConstant(True)
pars_d["alphaCB"].setConstant(True)

# Bkg only

total.Print("v")

# Create the NLL for the fit
nll = ROOT.RooNLLVar("nll","nll", total, ds_x, ROOT.RooFit.Extended(ROOT.kTRUE))
#fit_func = ROOT.RooFormulaVar("fit_func","nll + log_gc",ROOT.RooArgList(nll,pars_d["log_gc"]))

# Should I use RooAddition as mentioned here
# https://root.cern/doc/v612/classRooMinimizer.html
fit_func = ROOT.RooAddition("fit_func","sum of nll and log_gc",ROOT.RooArgList(nll,pars_d["log_gc"]))
# No gives same result with no asymmetric uncertainties

nll.setValueDirty()
fit_func.setValueDirty()

print("Setting up the minimizer..................................")
m = ROOT.RooMinimizer(fit_func)
#m = ROOT.RooMinimizer(nll)

# For NLL?
m.setErrorLevel(0.5)


m.setVerbose(ROOT.kTRUE)
m.setStrategy(2)

print("Helping get the initial step size.........................")
m.hesse()

print("Performing the minimizations..............................")
m.migrad()

print("Calculating the uncertainties..............................")
m.hesse()

# Does this matter?
#m.migrad()
#m.hesse()

fit_results = m.save()


fit_results.Print("v")





In [ ]:
#del c
c4 = ROOT.TCanvas("testingfits4", "testingfits4", 600, 300)#"fit", "fit", 600, 600)
# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe4 = x.frame(ROOT.RooFit.Title("extended ML fit example"), ROOT.RooFit.Bins(25))
ds_x.plotOn(xframe4)
total.plotOn(xframe4, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(sub_funcs['bkg_pdf'])
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_sig = ROOT.RooArgSet(sub_funcs['sig_pdf'])
total.plotOn(
    xframe4, ROOT.RooFit.Components(ras_sig), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.LineColor("r") , ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Draw the frame on the canvas

ROOT.gPad.SetLeftMargin(0.15)
xframe4.GetYaxis().SetTitleOffset(1.4)
xframe4.Draw()



#c.Update()
#ROOT.gPad
c4.Draw()
#ROOT.gPad.Draw()
#ROOT.gPad.Update()
#c.Draw()

c4.SaveAs("testfit_BNC.png")



In [ ]:
c5 = ROOT.TCanvas("testingfits5", "testingfits5", 600, 300)#"fit", "fit", 600, 600)
#frame1 = mean.frame(Bins=10, Range=(-0.5, 0.5), Title="LL and profileLL in mean")

#frame1 = pars_d['branching_fraction'].frame(Bins=10, Range=(0.0, 1.5), Title="LL and profileLL in mean")
frame1 = pars_d['branching_fraction'].frame(Bins=10, Range=(0.0, 5e-7), Title="LL and profileLL in mean")
nll.plotOn(frame1, ShiftToZero=True)

pll_conv = nll.createProfile({pars_d['branching_fraction']})
pll_conv.plotOn(frame1, LineColor="r")

# Adjust frame maximum for visual clarity
frame1.SetMinimum(0)
#frame1.SetMaximum(0.1)

c5.cd()

line = ROOT.TLine(0, 0.5, 1.5, 0.5)
line.SetLineColor(ROOT.kBlack)
line.SetLineWidth(2)

frame1.Draw()
c5.Draw()
line.Draw()

In [ ]:
dx = 1e-7

vals = []

for r in np.arange(0.0, 5e-7, 1e-8):

    pars_d['branching_fraction'].setVal(r)
    v = pll_conv.getVal()
    #print(f'{r:.2f}    {v:.4f}')
    if np.abs(v - 0.5) < dx:
        vals.append(r)

vals = np.array(vals)

print(vals)

central_val = 0.31497

print(vals - central_val)

# 0.16 3.15 0.525

In [ ]:
llhood = ROOT.RooFormulaVar("llhood","exp(-0.5*fit_func)", ROOT.RooArgList(fit_func));

#pll_conv = nll.createProfile({pars_d['conv_factor_fit']})

#llhoodP = ROOT.RooFormulaVar("llhoodP","exp(-0.5*pll_conv)", ROOT.RooArgList(pll_conv));

llhoodN = ROOT.RooFormulaVar("llhoodN","exp(-0.5*fit_func)",ROOT.RooArgList(fit_func));

c6 = ROOT.TCanvas("testingfits6", "testingfits6", 600, 300)#"fit", "fit", 600, 600)

frame3 = pars_d['branching_fraction'].frame(Bins=10, Range=(0.0, 5e-6), Title="LL and profileLL in mean")

llhoodN.plotOn(frame3)

frame3.Draw()
c6.Draw()

In [ ]:
pars_d['branching_fraction'].setVal(1.38e-7)
nll_min = nll.getVal()

pars_d['branching_fraction'].setVal(0)
nll_0 = nll.getVal()

print(f'nll_min: {nll_min}')
print(f'nll_0: {nll_0}')

sigma = np.sqrt(2*(nll_0-nll_min))
print(f"sigma: {sigma}")

In [ ]:
pars_d['branching_fraction'].setRange("IntRange1",0,5e-6)

total_int_val = llhoodN.createIntegral(ROOT.RooArgSet(pars_d['branching_fraction']),ROOT.RooFit.Range('IntRange1'))

print(total_int_val.getVal())

total_int_val = total_int_val.getVal()

xpts = []
ypts = []

for r in np.arange(0.0, 5e-6, 1e-8):
    
    pars_d['branching_fraction'].setRange("IntRange1",0,r)
    
    int_val = llhoodN.createIntegral(ROOT.RooArgSet(pars_d['branching_fraction']),ROOT.RooFit.Range('IntRange1'))
    
    #print(f"{r:.2e}  {int_val.getVal() / total_int_val:.4f}")
    xpts.append(r)
    ypts.append(int_val.getVal() / total_int_val)
;

plt.plot(xpts,ypts)
plt.plot([0,5e-6],[0.90, 0.90],'k--')
print(len(xpts))
print(xpts[-10:])

In [ ]:
#pll_conv.getAnalyticalIntegral()

In [ ]:
#pll_mean.Print('v')

In [ ]:
fit_func.defaultErrorLevel()

In [ ]:
pars_d['log_gc'].getVal()

In [ ]:
fit_results.Print("v")

In [ ]:
fit_results.Print("v")
print()
mynsig = pars_d["conv_factor_fit"].getVal() * pars_d["branching_fraction"].getVal()
print(f"mynsig: {mynsig}    nsig: {pars_d['nsig'].getVal()}")



In [ ]:
fit_results.Print('v')

In [ ]:
p = pars_d['conv_factor_fit']

print(p.getErrorLo(), p.getErrorHi())

In [ ]:
ROOT.TRolke

In [ ]:
mc = fit_func
data = ds_x

#pl = ROOT.RooStats.ProfileLikelihoodCalculator(data, mc)


In [ ]:
#teststat = ROOT.RooStats.SimpleLikelihoodRatioTestStat(sb_model.GetPdf(), b_model.GetPdf())
teststat = ROOT.RooStats.SimpleLikelihoodRatioTestStat(total, sub_funcs['bkg_pdf'])

#teststat.SetNullParameters(total.GetSnapshot())
#teststat.SetAltParameters(sub_functions['bkg_pdf'].GetSnapshot())

#hc2 = ROOT.RooStats.HybridCalculator(ds_x, sub_funcs['bkg_pdf'], total)





# RooMinimizer example with Hesse

In [ ]:
x = ROOT.RooRealVar ("x", "x", -20, 20)
mean = ROOT.RooRealVar ("mean", "mean", 0, -10, 10)
sigma = ROOT.RooRealVar ("sigma", "width", 3, 1, 10)

# Define a Gaussian PDF
gaus = ROOT.RooGaussian("gauss", "gaussian PDF", x, mean, sigma)

# Create a dataset 
data = gaus.generate(x, 1000)

# Create a negative log likelihood function from the PDF
nll = gaus.createNLL(data)

# 2. Create RooMinimizer
m = ROOT.RooMinimizer(nll)

# 3. Set the minimization strategy
m.setStrategy(1)

# 4. Call migrad() to minimize
m.migrad();

# 5. Call hesse() to perform error analysis
m.hesse()

# 6. Print the results
#print(f"Parameter values:")
#mean.Print()
#sigma.Print()
#print()

results = m.save()
print("Results ---------------------------")
results.Print("v")


In [ ]:
# Make canvas and draw ROOT.RooPlots
c = ROOT.TCanvas("rf605_profilell", "rf605_profilell", 800, 400)

c.Divide(2)

# Plot likelihood scan frac
frame1 = mean.frame(Bins=10, Range=(-0.5, 0.5), Title="LL and profileLL in mean")
nll.plotOn(frame1, ShiftToZero=True)
 
# Plot likelihood scan in sigma_g2
frame2 = sigma.frame(Bins=10, Range=(2.8, 3.2), Title="LL and profileLL in sigma_g2")
nll.plotOn(frame2, ShiftToZero=True)

In [ ]:
# The profile likelihood estimator on nll for frac will minimize nll w.r.t
# all floating parameters except frac for each evaluation
 
pll_mean = nll.createProfile({mean})
 
# Plot the profile likelihood in frac
pll_mean.plotOn(frame1, LineColor="r")
 
# Adjust frame maximum for visual clarity
frame1.SetMinimum(0)
frame1.SetMaximum(3)

In [ ]:

 

 
# Adjust frame maximum for visual clarity
frame2.SetMinimum(0)
frame2.SetMaximum(3)
 

c.cd(1)
ROOT.gPad.SetLeftMargin(0.15)
nll.plotOn(frame1, ShiftToZero=True)
# The profile likelihood estimator on nll for sigma_g2 will minimize nll
# w.r.t all floating parameters except sigma_g2 for each evaluation
pll_sigma = nll.createProfile({sigma})
frame1.GetYaxis().SetTitleOffset(1.4)
frame1.Draw()

c.cd(2)
ROOT.gPad.SetLeftMargin(0.15)
nll.plotOn(frame2, ShiftToZero=True)
# Plot the profile likelihood in sigma_g2
pll_sigma.plotOn(frame2, LineColor="r")

frame2.GetYaxis().SetTitleOffset(1.4)
frame2.Draw()

c.Draw()

In [ ]:
ROOT.__version__